<a href="https://colab.research.google.com/github/ChaeyiLee/git-workshop/blob/main/MLC_Animal_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from sklearn.utils import class_weight

In [5]:
def prepocess(image, label):
  return tf.image.resize(image, (64, 64)) / 255.0, label

train_data, test_data = tfds.load("cats_vs_dogs", split=["train[:80%]", "train[80%:]"], as_supervised = True)
train_data = train_data.map(prepocess).batch(32).prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(prepocess).batch(32).prefetch(tf.data.AUTOTUNE)

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
labels = np.concatenate([y for x, y in train_data], axis=0)
class_weights = dict(enumerate(class_weight.compute_class_weight('balanced', classes = np.unique(labels), y = labels)))

In [12]:
model.fit(train_data, validation_data = test_data, epochs = 2, class_weight = class_weights)

Epoch 1/2
582/582 ━━━━━━━━━━━━━━━━━━━━ 84s 140ms/step - accuracy: 0.5350 - loss: 0.6893 - val_accuracy: 0.6118 - val_loss: 0.6511
Epoch 2/2
582/582 ━━━━━━━━━━━━━━━━━━━━ 106s 181ms/step - accuracy: 0.6128 - loss: 0.6507 - val_accuracy: 0.6296 - val_loss: 0.6402


In [13]:
from tensorflow.keras.preprocessing import image

image_path = ""
img = image.load_img(image_path, target_size=(64, 64))
img_array = np.expand_dims(image.img_to_array(img), axis=0) / 255.0

prediction = model.predict(img_array)
label = np.argmax(prediction)

if label == 0:
  print("DOG")
else:
  print("CAT")

FileNotFoundError: [Errno 2] No such file or directory: ''